# Train Data - Average and High
Using code from code notebooks 00_1_garmin_gpx.ipynb & 00_2_openweather_api.ipynb to build out the two training datasets for female average performance and high performance. See code notebooks for sources on code. 

---
---

# Imports

In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import time

# GPX file
import gpxpy
import gpxpy.gpx

# GPX file
from bs4 import BeautifulSoup
import lxml

# bearing
from geographiclib.geodesic import Geodesic

# distance
import haversine as hs

# API
import requests

%matplotlib inline
pd.set_option('display.max_columns', None)

import warnings
warnings.simplefilter("ignore")

# Functions
---

## Historical Weather

In [2]:
def hist_weather(unix_datetime, lat, lon):
    '''
    Using the OpenWeather OneCall 3.0 API to pull historical daily data
    Input: UNIX datetime for day requesting
    Output: Pandas dataframe for that day's weather
    '''
    
    url_hist_point = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={unix_datetime}&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
    hist_point_req = requests.get(url_hist_point)

    wd_hist = hist_point_req.json()

    wd_hist_df = json_normalize(wd_hist['data'])
    wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
           'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

    
    return wd_hist_df

## GPX to Dataframe

In [3]:
def gpx_to_df(filepath):
    
    # Open .gpx file and parse xml
    with open(filepath, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        
    # check length of tracks
    print(f'Tracks: {len(gpx.tracks)}')
    # check number of data points
    print(f'No. of Track Points: {gpx.get_track_points_no()}')
    
    # extract timestamp, lat, lon, and elevation from gpx file
    route_info = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'timestamp': point.time,
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                })
    
    # create dataframe
    route_df = pd.DataFrame(route_info)
    # create UNIX column for later
    # https://statisticsglobe.com/convert-datetime-to-unix-timestamp-python
    route_df['dt'] = route_df['timestamp'].apply(lambda z: int(datetime.datetime.timestamp(z)))
    
    #######################
    # HEART RATE AND CADENCE
    # add extension data with beautifulsoup
    xml = gpx.to_xml()
    soup = BeautifulSoup(xml, features = 'xml')
    
    # Use Beautiful Soup to find all heart rate and cadence
    hr_soup = soup.find_all('ns3:hr')
    cad_soup = soup.find_all('ns3:cad')
    
    # chop off early points from index - accounts for before cadence sensor calibration
    route_df = route_df[-len(cad_soup):]

    # for loop to extract heart rate and cadence values from each point
    hr_list = []
    cad_list = []
    for i in hr_soup:
        # https://stackoverflow.com/questions/69420686/bs4-element-resultset-elements-to-a-list
        hr_list.append(i.get_text(strip = True))

    for j in cad_soup:
        cad_list.append(j.get_text(strip = True))

    # add columns to dataset from extraction
    route_df['heart_rate'] = hr_list[-len(cad_soup):]
    route_df['cadence'] = cad_list
    
    #######################
    # BEARING
    # get_bearing function
    def get_bearing(lat1, lat2, long1, long2):
        brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
        return brng
    
    # assign bearing in df
    route_df['bearing'] = 0
    for i in range(1, len(route_df)):
        lat1 = route_df.latitude.iloc[i-1]
        lat2 = route_df.latitude.iloc[i]
        long1 = route_df.longitude.iloc[i-1]
        long2 = route_df.longitude.iloc[i]
        bearing = get_bearing(lat1, lat2, long1, long2)
        # https://stats.stackexchange.com/questions/283572/using-iloc-to-set-values
        route_df.bearing.iloc[[i]] = bearing
    
    ########################    
    # ELAPSED TIME - SECONDS
    route_df['timestamp'] = pd.to_datetime(route_df['timestamp'])
    route_df['time_diff_s'] = 0
    for i in range(1, len(route_df)-1):
        t1 = route_df.iloc[i]['timestamp']
        t2 = route_df.iloc[i-1]['timestamp']
        # https://www.geeksforgeeks.org/how-to-set-cell-value-in-pandas-dataframe/
        route_df.at[i, 'time_diff_s'] = (t1 - t2).seconds
    route_df['total_time_s'] = route_df['time_diff_s'].cumsum(skipna = True)
    
    #######################
    # ELEVATION CHANGE - METERS
    route_df['ele_diff_m'] = 0
    for i in range(1, len(route_df)-1):
        e1 = route_df.iloc[i]['elevation']
        e2 = route_df.iloc[i-1]['elevation']
        route_df.at[i, 'ele_diff_m'] = (e1 - e2)
    route_df['total_ele_change_m'] = round(route_df['ele_diff_m'].cumsum(skipna = True), 4)
    
    #######################
    # DISTANCE - KILOMETERS
    route_df['lat_lon'] = [(lat, lon) for lat, lon in zip(route_df['latitude'], route_df['longitude'])]
    route_df['dist_diff_km'] = 0
    for i in range(1, len(route_df)-1):
        loc1 = route_df.iloc[i]['lat_lon']
        loc2 = route_df.iloc[i-1]['lat_lon']
        route_df.at[i, 'dist_diff_km'] = round(hs.haversine(loc1, loc2), 4) # kilometers
    route_df['total_dist_km'] = route_df['dist_diff_km'].cumsum(skipna = True)
    
   
    #######################
    route_df.reset_index(drop = True, inplace = True)
    
    return route_df

# Average Cycling Dataset

## Get GPX files and turn to dataframes in a dictionary

In [4]:
#https://stackoverflow.com/questions/13603215/using-a-loop-in-python-to-name-variables
average = dict()

for x in range(1, 13):
    average[x] = gpx_to_df(f'../data/average/a_{x}.gpx')

Tracks: 1
No. of Track Points: 400
Tracks: 1
No. of Track Points: 3015
Tracks: 1
No. of Track Points: 853
Tracks: 1
No. of Track Points: 1333
Tracks: 1
No. of Track Points: 157
Tracks: 1
No. of Track Points: 2224
Tracks: 1
No. of Track Points: 1364
Tracks: 1
No. of Track Points: 1070
Tracks: 1
No. of Track Points: 739
Tracks: 1
No. of Track Points: 181
Tracks: 1
No. of Track Points: 1453
Tracks: 1
No. of Track Points: 2304


## Get historical weather and join with dataframes

In [5]:
average[1]

,timestamp,latitude,longitude,elevation,dt,heart_rate,cadence,bearing,time_diff_s,total_time_s,ele_diff_m,total_ele_change_m,lat_lon,dist_diff_km,total_dist_km
0,2022-07-20 16:07:45+00:00,38.773466,-121.363686,35.799999,1658333265,78,0,0.000000,0,0,0.000000,0.0,"(38.77346634864807, -121.36368582956493)",0.0000,0.0000
1,2022-07-20 16:07:46+00:00,38.773542,-121.363672,35.599998,1658333266,79,0,8.292053,1,1,-0.200001,-0.2,"(38.77354153431952, -121.36367183178663)",0.0084,0.0084
2,2022-07-20 16:07:49+00:00,38.773630,-121.363682,35.200001,1658333269,82,0,-5.321180,3,4,-0.399998,-0.6,"(38.77363029867411, -121.36368239298463)",0.0099,0.0183
3,2022-07-20 16:07:51+00:00,38.773789,-121.363733,35.000000,1658333271,83,0,-13.956066,2,6,-0.200001,-0.8,"(38.77378871664405, -121.36373268440366)",0.0181,0.0364
4,2022-07-20 16:07:52+00:00,38.773786,-121.363766,35.000000,1658333272,83,0,-96.936537,1,7,0.000000,-0.8,"(38.77378553152084, -121.36376612819731)",0.0029,0.0393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2022-07-20 16:38:30+00:00,38.773919,-121.363615,27.600000,1658335110,78,34,-150.452516,2,1845,0.000000,-8.2,"(38.77391922287643, -121.36361475102603)",0.0047,4.5232
396,2022-07-20 16:38:34+00:00,38.773825,-121.363649,27.600000,1658335114,81,0,-163.970071,4,1849,0.000000,-8.2,"(38.773824675008655, -121.36364945210516)",0.0109,4.5341
397,2022-07-20 16:38:36+00:00,38.773789,-121.363649,27.799999,1658335116,77,0,179.363825,2,1851,0.199999,-8.0,"(38.77378921955824, -121.36364894919097)",0.0039,4.5380
398,2022-07-20 16:38:37+00:00,38.773776,-121.363640,28.000000,1658335117,77,0,153.254669,1,1852,0.200001,-7.8,"(38.773775808513165, -121.36364031583071)",0.0017,4.5397


In [6]:
average[1]['latitude'][0]

38.77346634864807

In [7]:
average[1]['dt'][1]

1658333266

In [ ]:
average[1].dropna(inplace = True)
a1_weather = pd.DataFrame()
for i in range(len(average[1])):
    unix = average[1]['dt'][i]
    lat = average[1]['latitude'][i]
    lon = average[1]['longitude'][i]
    weather = hist_weather(unix, lat, lon)
    a1_weather.append(weather)